## This notebook requires TensorFlow 2.0 (or later)!

In [1]:
# Update this folder name for your computer

local_data_folder = "d:\Data\SagittalSpineSegmentationStudy"

overwrite_existing_data_files = False

# Learning parameters

import numpy as np

ultrasound_size = 128
num_classes = 2
num_epochs = 5
batch_size = 24
max_learning_rate = 0.02
min_learning_rate = 0.00001
regularization_rate = 0.0001
WCE_weights = np.array([0.05, 0.95])
learning_rate_decay = (max_learning_rate - min_learning_rate) / num_epochs

In [2]:
import datetime
import os 

from ipywidgets import IntProgress
from IPython.display import display

import girder_client
import matplotlib.pyplot as plt
import tensorflow as tf

import ultrasound_batch_generator
import sagittal_spine_segmentation_unet

In [3]:
# Define what data to download

girder_api_url = "https://pocus.cs.queensu.ca/api/v1"

training_ultrasound_ids = [
    "5da9e5c0d9e6a3be02d012b4",
    "5da9e5c7d9e6a3be02d012c6",
    "5da9e5c2d9e6a3be02d012b7",
    "5da9e5c3d9e6a3be02d012ba",
    "5da9e5c8d9e6a3be02d012c9",
    "5da9e5c5d9e6a3be02d012c0",
    "5da9e5c6d9e6a3be02d012c3",
    "5da9e5c4d9e6a3be02d012bd"
]

training_ultrasound_filenames = [
    "q000_ultrasound.npy",
    "q001_ultrasound.npy",
    "q002_ultrasound.npy",
    "q003_ultrasound.npy",
    "q004_ultrasound.npy",
    "q005_ultrasound.npy",
    "q006_ultrasound.npy",
    "q007_ultrasound.npy"
]

training_segmentation_ids = [
    "5da9e5c8d9e6a3be02d012cc",
    "5da9e5ccd9e6a3be02d012de",
    "5da9e5c9d9e6a3be02d012cf",
    "5da9e5cad9e6a3be02d012d2",
    "5da9e5cdd9e6a3be02d012e1",
    "5da9e5cbd9e6a3be02d012d8",
    "5da9e5cbd9e6a3be02d012db",
    "5da9e5cad9e6a3be02d012d5"
]

training_segmentation_filenames = [
    "q000_segmentation.npy",
    "q001_segmentation.npy",
    "q002_segmentation.npy",
    "q003_segmentation.npy",
    "q004_segmentation.npy",
    "q005_segmentation.npy",
    "q006_segmentation.npy",
    "q007_segmentation.npy"
]

testing_ultrasound_filename = "ultrasound-test.npy"
testing_ultrasound_id = "5daa85edd9e6a3be02d012e7"
testing_segmentation_filename = "segmentation-test.npy"
testing_segmentation_id = "5daa85e7d9e6a3be02d012e4"

In [4]:
# These subfolders will be created/populated in the data folder

data_arrays_folder    = "DataArrays"
notebooks_save_folder = "SavedNotebooks"
models_save_folder    = "SavedModels"

data_arrays_fullpath = os.path.join(local_data_folder, data_arrays_folder)
notebooks_save_fullpath = os.path.join(local_data_folder, notebooks_save_folder)
models_save_folder = os.path.join(local_data_folder, models_save_folder)

if not os.path.exists(data_arrays_fullpath):
    os.makedirs(data_arrays_fullpath)
    print("Created folder: {}".format(data_arrays_fullpath))

if not os.path.exists(notebooks_save_fullpath):
    os.makedirs(notebooks_save_fullpath)
    print("Created folder: {}".format(notebooks_save_fullpath))

if not os.path.exists(models_save_folder):
    os.makedirs(models_save_folder)
    print("Created folder: {}".format(models_save_folder))

In [5]:
# Download data from Girder

time_download_start = datetime.datetime.now()

print("Downloading training files ...")

n_files = len(training_ultrasound_ids)

f = IntProgress(min=0, max=n_files*2)
display(f)

gclient = girder_client.GirderClient(apiUrl=girder_api_url)

for i in range(n_files):
    ultrasound_fullname = os.path.join(data_arrays_fullpath, training_ultrasound_filenames[i])
    if not os.path.exists(ultrasound_fullname) or overwrite_existing_data_files:
        print("Downloading {}...".format(ultrasound_fullname))
        gclient.downloadFile(training_ultrasound_ids[i], ultrasound_fullname)
    f.value = i * 2 + 1
    
    segmentation_fullname = os.path.join(data_arrays_fullpath, training_segmentation_filenames[i])
    if not os.path.exists(segmentation_fullname) or overwrite_existing_data_files:
        print("Downloading {}...".format(segmentation_fullname))
        gclient.downloadFile(training_segmentation_ids[i], segmentation_fullname)
    f.value = i * 2 + 2

test_ultrasound_fullname = os.path.join(data_arrays_fullpath, testing_ultrasound_filename)
if not os.path.exists(test_ultrasound_fullname):
    print("Downloading {}...".format(test_ultrasound_fullname))
    gclient.downloadFile(testing_ultrasound_id, test_ultrasound_fullname)

test_segmentation_fullname = os.path.join(data_arrays_fullpath, testing_segmentation_filename)
if not os.path.exists(test_segmentation_fullname) or overwrite_existing_data_files:
    print("Downloading {}...".format(test_segmentation_fullname))
    gclient.downloadFile(testing_segmentation_id, test_segmentation_fullname)
    
time_download_stop = datetime.datetime.now()
print("\nTotal download time: {}".format(time_download_stop - time_download_start))

IntProgress(value=0, max=16)


Total download time: 0:00:00.020945


In [6]:
# Read data into numpy arrays

ultrasound_arrays = []
segmentation_arrays = []

f = IntProgress(min=0, max=n_files * 2)
display(f)

time_start = datetime.datetime.now()

for i in range(n_files):
    ultrasound_fullname = os.path.join(data_arrays_fullpath, training_ultrasound_filenames[i])
    segmentation_fullname = os.path.join(data_arrays_fullpath, training_segmentation_filenames[i])

    ultrasound_data = np.load(ultrasound_fullname)
    f.value = i * 2 + 1
    
    segmentation_data = np.load(segmentation_fullname)
    f.value = i * 2 + 2
    
    ultrasound_arrays.append(ultrasound_data)
    segmentation_arrays.append(segmentation_data)

test_ultrasound_fullname = os.path.join(data_arrays_fullpath, testing_ultrasound_filename)
test_ultrasound_array = np.load(test_ultrasound_fullname)

test_segmentation_fullname = os.path.join(data_arrays_fullpath, testing_segmentation_filename)
test_segmentation_array = np.load(test_segmentation_fullname)
    
time_stop = datetime.datetime.now()
print("\nTotal time to load from files: {}".format(time_stop - time_start))

IntProgress(value=0, max=16)


Total time to load from files: 0:00:00.130138


In [7]:
# Print data sizes

print("Training data")
for i in range(n_files):
    print("ultr {} : {}".format(i, ultrasound_arrays[i].shape))
    print("segm {} : {}".format(i, segmentation_arrays[i].shape))

print("\nTesting data")
print("test ultr : {}".format(test_ultrasound_array.shape))
print("test segm : {}".format(test_segmentation_array.shape))

Training data
ultr 0 : (523, 128, 128, 1)
segm 0 : (523, 128, 128, 1)
ultr 1 : (355, 128, 128, 1)
segm 1 : (355, 128, 128, 1)
ultr 2 : (477, 128, 128, 1)
segm 2 : (477, 128, 128, 1)
ultr 3 : (453, 128, 128, 1)
segm 3 : (453, 128, 128, 1)
ultr 4 : (289, 128, 128, 1)
segm 4 : (289, 128, 128, 1)
ultr 5 : (387, 128, 128, 1)
segm 5 : (387, 128, 128, 1)
ultr 6 : (360, 128, 128, 1)
segm 6 : (360, 128, 128, 1)
ultr 7 : (446, 128, 128, 1)
segm 7 : (446, 128, 128, 1)

Testing data
test ultr : (1892, 128, 128, 1)
test segm : (1892, 128, 128, 1)


In [8]:
# Print training parameters, to archive them together with the notebook output.

time_sequence_start = datetime.datetime.now()
print("Training started at: {}".format(time_sequence_start))
print("Number of epochs:    {}".format(num_epochs))
print("Step size maximum:   {}".format(max_learning_rate))
print("Step size decay:     {}".format(learning_rate_decay))
print("Batch size:          {}".format(batch_size))
print("Regularization rate: {}".format(regularization_rate))

for i in range(n_files):
    
    # Prepare data arrays
    
    train_ultrasound_data = np.zeros(
        [0, ultrasound_arrays[0].shape[1], ultrasound_arrays[0].shape[2], ultrasound_arrays[0].shape[3]])
    train_segmentation_data = np.zeros(
        [0, ultrasound_arrays[0].shape[1], ultrasound_arrays[0].shape[2], ultrasound_arrays[0].shape[3]])
    
    val_ultrasound_data = ultrasound_arrays[i]
    val_segmentation_data = segmentation_arrays[i]
    
    for train_index in range(n_files):
        if train_index != i:
            train_ultrasound_data = np.concatenate((train_ultrasound_data, ultrasound_arrays[train_index]))
            train_segmentation_data = np.concatenate((train_segmentation_data, segmentation_arrays[train_index]))
    
    print("")
    print("Validation ultrasound shape:   {}".format(train_ultrasound_data.shape))
    print("Validation segmentation shape: {}".format(train_segmentation_data.shape))
    
    # Create and train model
    
    model = sagittal_spine_segmentation_unet.sagittal_spine_unet(ultrasound_size, num_classes, regularization_rate)
    model.compile(optimizer=keras.optimizers.adam(lr=max_learning_rate, decay=learning_rate_decay),
              loss=[weighted_categorical_crossentropy(WCE_weights)],
              metrics=["accuracy", dice_coef])
    
    training_generator = UltrasoundSegmentationBatchGenerator(
        train_ultrasound_data, train_segmentation_data[:, :, :, 0], batch_size)
    
    validation_generator = UltrasoundSegmentationBatchGenerator(
        val_ultrasound_data, val_segmentation_data[:, :, :, 0], batch_size)
    
    training_time_start = datetime.datetime.now()
    
    training_log = model.fit_generator(training_generator,
                                       validation_data=validation_generator,
                                       epochs=num_epochs,
                                       verbose=0)
        
    training_time_stop = datetime.datetime.now()
    


time_sequence_stop = datetime.datetime.now()

print("\nAll training stopped at: {}".format(time_sequence_stop))
print("\nTotal training time:        {}".format(time_sequence_stop - time_sequence_start))


Training started at: 2019-10-19 03:18:55.998311
Number of epochs:    5
Step size maximum:   0.02
Step size decay:     0.003998
Batch size:          24
Regularization rate: 0.0001

Validation ultrasound shape:   (2767, 128, 128, 1)
Validation segmentation shape: (2767, 128, 128, 1)


InternalError: cudaGetDevice() failed. Status: cudaGetErrorString symbol not found.

In [ ]:
print("\n")